# Knowledge Representation on the Web -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [47]:
import sys
# !{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef, Namespace, Literal
from rdflib.namespace import FOAF

from owlrl import DeductiveClosure, RDFS_Semantics

## Loading RDFS graphs

Your file `yourRDF.ttl` already contains a basic Knowledge Graph in RDF with some RDFS semantics

First, we are going to add some RDFS semantics, and inspect the graph as-is; this is also called the "asserted graph"

**Exercise 1** 
1. add additional triples using the RDFS semantics: have a look [here](https://www.w3.org/TR/rdf-schema/), and use domain and range, subPropertyOf, and Class, to say more about the instances in your graph
2. load yourRDF graph
3. print the classes in your graph
4. print the properties of a specific class in yourRDF graph
5. print all instances in yourRDF graph (all objects that have a type) 
6. explain what constitutes a vocabulary in RDF

In [21]:
#define graph
g = Graph()

g = g.parse("./data/myRDF.ttl")

# print(g.serialize(format='ttl'))

#define example namespace

TEACH = Namespace("http://linkedscience.org/teach/ns#")
KRW = Namespace("http://krw.vu.nl/data#")
CON = Namespace("http://www.w3.org/2000/10/swap/pim/contact#")
g.bind("contact", CON)

# g.namespace_manager.bind('CON', URIRef('http://www.w3.org/2000/10/swap/pim/contact#'))
bob = URIRef("http://example.org/people/Bob")
linda = URIRef("http://example.org/people/Linda")  # a GUID is generated


#Add triples using store's add method.

g.add((bob, RDF.type, CON.Male))
g.add((CON.Male, RDFS.subClassOf, FOAF.Person))

g.add((linda, RDF.type, CON.Female))
g.add((CON.Female, RDFS.subClassOf, FOAF.Person))

#print classes

for s, p, o in g.triples((None, RDF.type, None)):
    print(o)
# print('help')
# print(g.serialize(format='ttl'))

#print properties
for s, p, o in g.triples((linda, None, None)):
    print(p)



for s in g.subjects(RDF.type, None):
    print(s)
    #print all instances




http://krw.vu.nl/data#Student
http://xmlns.com/foaf/0.1/Person
http://xmlns.com/foaf/0.1/Person
http://krw.vu.nl/data#Teacher
http://krw.vu.nl/data#Course
http://www.w3.org/2000/10/swap/pim/contact#Male
http://www.w3.org/2000/10/swap/pim/contact#Female
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://krw.vu.nl/data#teacherOf
http://xmlns.com/foaf/0.1/age
http://xmlns.com/foaf/0.1/name
http://example.org/people/Bob
http://example.org/people/Bob
http://example.org/people/Linda
http://example.org/people/Linda
nb13fc22a7fc34b618a29b4c2b529ddcbb1
http://example.org/people/Bob
http://example.org/people/Linda


## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


**Exercise 2**
1. expand the graph through RDFS semantics inference
2. print how many triples the new graph has
3. print out the triples in your new graph and inspect them. 

In [25]:
DeductiveClosure(RDFS_Semantics).expand(g)

print(len(g))

print(g.serialize(format='ttl'))
#TIP:always look at linked documentation 

61
@prefix contact: <http://www.w3.org/2000/10/swap/pim/contact#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix teach: <http://krw.vu.nl/data#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a teach:Student,
        rdfs:Resource,
        contact:Male,
        foaf:Person ;
    foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda> a teach:Teacher,
        rdfs:Resource,
        contact:Female,
        foaf:Person ;
    teach:teacherOf [ a teach:Course,
                rdfs:Resource ;
            teach:hasDescription "Weekly fun stuff" ;
            teach:hasTitle "Knowledge Representation on the Web" ] ;
    foaf:age 40 ;
    foaf:name "Linda" .

teach:Course a rdfs:Resource .

teach:Student a rdfs:Resource .

teach:Teacher a rdfs:Resource .

teach:hasDescr

## The explicit (asserted) graph vs the implicit (derived) graph, and RDF entailment

Asserted triples are those that are explicitly stated, while derived or inferred triples are those that are implicitly stated through the semantics of RDFS. 

**Exercise 3**

1. Write here code to generate a graph that contains **RDFS derived triples only** from yourRDF Knowledge Graph, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)
2. have a look at the inferred graph. Based on the RDFS semantics, explain for each triple the rule that was used to generate it.
3. Explain the concept RDF entailment, and the types of entailment RDFS can produce


In [26]:
g1 = Graph()

g1.parse('./data/myRDF.ttl')

g2 = g - g1

print(g2.serialize(format='ttl'))

@prefix ns1: <http://krw.vu.nl/data#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a rdfs:Resource,
        <http://www.w3.org/2000/10/swap/pim/contact#Male> .

<http://example.org/people/Linda> a rdfs:Resource,
        <http://www.w3.org/2000/10/swap/pim/contact#Female> ;
    ns1:teacherOf [ a ns1:Course,
                rdfs:Resource ;
            ns1:hasDescription "Weekly fun stuff" ;
            ns1:hasTitle "Knowledge Representation on the Web" ] .

ns1:Student a rdfs:Resource .

ns1:Teacher a rdfs:Resource .

24 a rdfs:Resource .

40 a rdfs:Resource .

"Bob" a rdfs:Resource .

"Linda" a rdfs:Resource .

ns1:Course a rdfs:Resource .

ns1:hasDescription a rdf:Property,
        rdfs:Resource ;
    rdfs:subPropertyOf ns1:hasDescription .

ns1:hasTitle a rdf:Property,
        rdfs:Resource ;
    rdfs:subPropertyOf ns1:hasTitle .

## Assignment part 2: your own webapplication. 


**Exercise 4**
1. load ingredients.rdf and recipes.rdf in one graph. The graph contains types of individuals and types of relationships between them. Print all the classes and properties in the combined graph with the namespace `ind` and the `wtm` namespace/vocabulary (`http://purl.org/heals/food/`). 

2. extend the `ind` vocabulary (`http://purl.org/heals/ingredient/`) by creating a hierarchy of ingredients (**hint: http://purl.org/heals/ingredient/CoconutMilk rdf:subClassOf http://purl.org/heals/ingredient/PlantMilk), and make these superclasses human readable by giving them labels**) 
3. do the same for the `wtm` vocabulary: add a hierarchy of recipes as well as a hierarchy of properties (**hint: http://purl.org/heals/food/hasCookingTemperature rdf:subPropertyOf ...) 
4. print the entailed triples as we did in the previous exercise
5. give three examples of how RDF semantics could aid the chefs in your restaurant 
    
6. which properties and classes could you add to the `wtm` and `ind` vocabularies to further describe your recipe and ingredient knowledge graph, aiding the chefs in your restaurant?  


In [58]:
#In order to extend the vocabularies you have to come up with multiple examples that follows the given hierarchy

g_food = Graph()

g_food.parse('./data/ingredients.rdf')
g_food.parse('./data/recipes.rdf')

IND = Namespace('http://purl.org/heals/ingredient/')
WTM = Namespace('http://purl.org/heals/food/')

classes = {o for _, _, o in g_food.triples((None, RDF.type, None)) if (o in IND or o in WTM)}

print('printing classes:')
for c in classes:
    print(c)

ind_wtm_properties = set()
for s, _, _ in g_food.triples((None, RDF.type, None)):
    properties = {p for _, p, _ in g_food.triples((s, None, None)) if (p in IND or p in WTM)}
#     print(properties)
    if len(properties)>0:
        ind_wtm_properties.update(properties)
print('\nprinting properties:')
for p in properties:
    print(p)


instances = {s for s, _, _ in g_food.triples((None, RDF.type, None)) if (s in IND or s in WTM)}
    
print('\nprinting instances')
for i in instances:
    print(i)

printing classes:
http://purl.org/heals/food/MeatFromCow
http://purl.org/heals/food/MeatFromLamb
http://purl.org/heals/food/ChemicalFoodProduct
http://purl.org/heals/food/TimeMeasurement
http://purl.org/heals/food/Yeast
http://purl.org/heals/food/Recipe
http://purl.org/heals/food/Flavor
http://purl.org/heals/food/Ingredient
http://purl.org/heals/food/MilkFromCow
http://purl.org/heals/food/User
http://purl.org/heals/food/MeatFromChicken
http://purl.org/heals/food/Texture
http://purl.org/heals/food/TemperatureMeasurement

printing properties:
http://purl.org/heals/food/hasCookingTemperature
http://purl.org/heals/food/hasCookTime
http://purl.org/heals/food/serves
http://purl.org/heals/food/isRecommendedForCourse
http://purl.org/heals/food/hasIngredient
http://purl.org/heals/food/isRecommendedForMeal

printing instances
http://purl.org/heals/ingredient/ChickenBroth
http://purl.org/heals/ingredient/Oat
http://purl.org/heals/ingredient/Clove
http://purl.org/heals/ingredient/ChickenEgg
http:/

In [59]:
g_food.add((IND.CoconutMilk, RDFS.subClassOf, IND.PlantMilk))
g_food.add((IND.PlantMilk, RDFS.label, Literal('Vegan alternative to cow milk')))

g_food.add((IND.PlantMilk, RDFS.subClassOf, IND.Milk))
g_food.add((IND.Milk, RDFS.label, Literal('White and cloudy liquid. Often used for making creamy dishes')))

g_food.add((IND.CowMilk, RDFS.subClassOf, IND.Milk))

g_food.add((WTM.MeatFromCow, RDFS.subClassOf, WTM.Meat))
g_food.add((WTM.MeatFromLamb, RDFS.subClassOf, WTM.Meat))
g_food.add((WTM.MeatFromChicken, RDFS.subClassOf, WTM.Meat))
g_food.add((WTM.MeatFromPig, RDFS.subClassOf, WTM.Meat))
g_food.add((WTM.MeatFromFish, RDFS.subClassOf, WTM.Meat))

g_food.add((WTM.Meat, RDFS.label, Literal('Meat from any type of animal')))
# print(g.serialize(format='ttl'))

<Graph identifier=Ndc402e3d5dcf4559a7b294f5e1322bd3 (<class 'rdflib.graph.Graph'>)>

In [60]:
g_food.add((IND.isRecommendedForCourse, RDFS.subPropertyOf, IND.isRecommendation))
g_food.add((IND.isRecommendation, RDFS.label, Literal("A suggestion for a cook on how to use the recipe or ingredient")))

g_food.add((WTM.procedure, RDF.type, RDF.Property))
g_food.add((WTM.hasCookTime, RDFS.subPropertyOf, WTM.procedure))
g_food.add((WTM.hasCookingTemperature, RDFS.subPropertyOf, WTM.procedure))

<Graph identifier=Ndc402e3d5dcf4559a7b294f5e1322bd3 (<class 'rdflib.graph.Graph'>)>

In [61]:
original = Graph()

original.parse('./data/ingredients.rdf')
original.parse('./data/recipes.rdf')

DeductiveClosure(RDFS_Semantics).expand(g_food)
g_food.bind("wtm", WTM)
g_food.bind("ind", IND)

new_triples = g_food - original
new_triples.bind("wtm", WTM)
new_triples.bind("ind", IND)


print(f'number of inferenced triples: {len(new_triples)}')
print(new_triples.serialize(format='ttl'))

number of inferenced triples: 933
@prefix ind: <http://purl.org/heals/ingredient/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wtm: <http://purl.org/heals/food/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://purl.obolibrary.org/obo/FOODON_03400662> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400663> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400666> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400667> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400670> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400671> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400673> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400674> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03400675> a rdfs:Resource .

<http://purl.obolibrary.org/obo/FOODON_03